# Amazon fine food review

In [ ]:
import pandas as pd
import spacy
from gensim.models import Word2Vec

## Loading data

In [ ]:
from datasets import load_dataset
dataset = load_dataset("jhan21/amazon-food-reviews-dataset")

In [8]:
# Load data from https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
df = pd.read_csv("../data/Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
# Load the English model
nlp = spacy.load("en_core_web_md")

# Function to tokenize and lowercase sentences
def tokenize_and_lowercase(x, column_name):
    text = x[column_name]
    doc = nlp(text)
    return [[token.text.lower() for token in sent if not token.is_stop and not token.is_punct] for sent in doc.sents]


In [ ]:
# Tokenize and lowercase the text and create list of tokenized sentences
df = df.sample(frac=0.5).copy()
df['tokenized'] = df.apply(lambda x: tokenize_and_lowercase(x, column_name='Text'), axis=1)

# Save the tokenized data
df.to_parquet('amazon-food-review-tokenized.parquet', engine='pyarrow')

In [4]:
# Load the tokenized data
df = pd.read_parquet('amazon-food-review-tokenized.parquet', engine='pyarrow')

# flatten the list of tokenized sentences
list_of_sentences = [list(tokens) for sublist in df['tokenized'].tolist() for tokens in sublist]

In [ ]:
# Create and train the Word2Vec model
model_food = Word2Vec(list_of_sentences, vector_size=100, window=5, min_count=5, workers=4, epochs=5, sg=1)

# Save the word2vec model
model_food.save("word2vec_food_review.model")

In [11]:
# Load the word2vec model
#model_fod = Word2Vec.load("word2vec_food_review.model")

# Example usage
similar_words = model_food.wv.most_similar('disgusting', topn=5)
print(similar_words)

[('awful', 0.7942688465118408), ('gross', 0.7863739132881165), ('nasty', 0.7592806220054626), ('vile', 0.7541845440864563), ('revolting', 0.7451091408729553)]
